In [1]:

import napari
import os
import glob
from pathlib import Path
from tifffile import imread
import numpy as np
import csv
import pandas as pd
from qtpy.QtWidgets import QComboBox, QPushButton, QSlider
%gui qt

In [2]:
class EventViewer(object):
    
    def __init__(self, viewer, imagename, Name, csv_dir, save = False, newimage = True):
     
          self.save = save
          self.newimage = newimage
          self.viewer = viewer
           
          print('reading image')      
          self.imagename = imagename  
          self.image = imread(imagename)
          print('image read')
          self.Name = Name
            
          self.csv_dir = csv_dir
          
          
          self.click()
    def click(self):

        
                ClassBackground = 'Normal'
                ClassDivision = 'Division'
                
                if self.save == True:
 
                        ClassBackgrounddata = self.viewer.layers[ClassBackground].data
                        bgdf = pd.DataFrame(ClassBackgrounddata, index = None, columns = ['T', 'Y', 'X'])
                        bgdf.to_csv(self.csv_dir + '/' + 'CAPED' + ClassBackground + self.Name +  '.csv', index = False, mode = 'w')

                        ClassDivisiondata = self.viewer.layers[ClassDivision].data
                        divdf = pd.DataFrame(ClassDivisiondata, index = None, columns = ['T', 'Y', 'X'])
                        divdf.to_csv(self.csv_dir + '/' +'CAPED' + ClassDivision + self.Name +  '.csv', index = False, mode = 'w')

                       
        
                if self.newimage == True:
                     for layer in list(self.viewer.layers):

                            self.viewer.layers.remove(layer) 
                    
                if self.save == False:
                        self.viewer.add_image(self.image, name = self.Name)


                        # add the first points layer for ClassBackground T point
                        self.viewer.add_points(name= ClassBackground, face_color='red', ndim = 3)

                        # add the second points layer for ClassDivision T point
                        self.viewer.add_points(name= ClassDivision, face_color='blue', ndim = 3)

                      
                        # programatically enter add mode for both Points layers to enable editing

                        self.viewer.layers[ClassBackground].mode = 'add'
                        self.viewer.layers[ClassDivision].mode = 'add'
                       

In [3]:
sourcedir = '/mnt/c/Users/rando/Downloads/TrainingDatasets/Hela_CAPED/'
csv_dir = '/mnt/c/Users/rando/Downloads/TrainingDatasets/Hela_CAPED/Csv/'
Imageids = []
Boxname = 'ImageIDBox'
Path(csv_dir).mkdir(exist_ok = True)



Raw_path = os.path.join(sourcedir, '*tif')
X = glob.glob(Raw_path)
for imagename in X:
         Imageids.append(imagename)
         Name = os.path.basename(os.path.splitext(imagename)[0])

imageidbox = QComboBox()   
imageidbox.addItem(Boxname)   
tracksavebutton = QPushButton('Save Clicks')

for i in range(0, len(Imageids)):


     imageidbox.addItem(str(Imageids[i]))
        
        
viewer = napari.Viewer()        
viewer.window.add_dock_widget(imageidbox, name="Image", area='bottom')    
viewer.window.add_dock_widget(tracksavebutton, name="Save Clicks", area='bottom')
imageidbox.currentIndexChanged.connect(
         lambda trackid = imageidbox: EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, False, True ))     

tracksavebutton.clicked.connect(
        lambda trackid= tracksavebutton:EventViewer(
                 viewer,
                  imageidbox.currentText(),
                       os.path.basename(os.path.splitext(imageidbox.currentText())[0]), csv_dir, True, False ))

DIPlib -- a quantitative image analysis library
Version 3.1.0 (Sep 24 2021)
For more information see https://diplib.org


reading image
image read
